# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

In [1]:
import pandas as pd
import numpy as np
from math import log
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('kc_house_data.csv')
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [3]:
train_data = pd.read_csv('kc_house_train_data.csv')
test_data = pd.read_csv('kc_house_test_data.csv')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
X = np.array(train_data[['sqft_living', 'bedrooms', 'bathrooms']])
Y = np.array(train_data['price'])

reg = LinearRegression()
model = reg.fit(X,Y)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
reg.coef_

array([   315.40669062, -65081.88711588,   6942.16598637])

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
example_predictions = reg.predict(X)
example_predictions[0] # should be 271789.505878

271789.26537996985

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [7]:
def get_residual_sum_of_squares(model, X, Y):
    
    '''
    Arguments :
    model -- the multiple linear regression model learned as above
    X -- the input features 'sqft_living', 'bedrooms', 'bathrooms' from the train_data
    Y -- the output values of 'price' from the train_data
    
    Returns :
    RSS -- the residual sum of squares, i.e. the difference between the predicted value and the actual value
    
    '''
    # First get the predictions
    predictions = np.array(model.predict(X))

    # Then compute the residuals/errors
    residuals = predictions - Y

    # Then square and add them up
    RSS = np.sum(np.square(residuals))

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [8]:
X_test = test_data[['sqft_living', 'bedrooms', 'bathrooms']]
Y_test = test_data['price']

rss_example_train = get_residual_sum_of_squares(model, X_test, Y_test)
print(rss_example_train) # should be 2.7376153833e+14

273761940583133.75


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [9]:
# Creating new features for the train_data
train_data['bedrooms_squared'] = train_data['bedrooms'] * train_data['bedrooms']
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
train_data['log_sqft_living'] = np.log10(train_data['sqft_living'])
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']

# Create new features for the test_data
test_data['bedrooms_squared'] = test_data['bedrooms'] * test_data['bedrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
test_data['log_sqft_living'] = np.log10(test_data['sqft_living'])
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

# The code below is used to answer the following quiz question

test_bedrooms_squared_mean = round(sum(test_data['bedrooms_squared']) / len(test_data['bedrooms_squared']),2)
test_bed_bath_rooms_mean = round(sum(test_data['bed_bath_rooms']) / len(test_data['bed_bath_rooms']),2)
test_log_sqft_living_mean = round(sum(test_data['log_sqft_living']) / len(test_data['log_sqft_living']),2)
test_lat_plus_long_mean = round(sum(test_data['lat_plus_long']) / len(test_data['lat_plus_long']),2)

print('Mean of bedrooms_squared for test data : ', test_bedrooms_squared_mean)
print('Mean of bed_bath_rooms for test data   : ', test_bed_bath_rooms_mean)
print('Mean of log_sqft_living for test data  : ', test_log_sqft_living_mean)
print('Mean of lat_plus_long for test data    : ', test_lat_plus_long_mean)

Mean of bedrooms_squared for test data :  12.45
Mean of bed_bath_rooms for test data   :  7.5
Mean of log_sqft_living for test data  :  3.28
Mean of lat_plus_long for test data    :  -74.65


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

<font color = 'steelblue'><b> Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits) </b></font>

<font color = 'mediumvioletred'><b> Answer  : The mean value of 4 new features on TEST data are : </b></font>
<font color = 'slategray'><b>
- Mean of bedrooms_squared for test data : {{test_bedrooms_squared_mean}}
- Mean of bed_bath_rooms for test data : {{test_bed_bath_rooms_mean}}
- Mean of log_sqft_living for test data : {{test_log_sqft_living_mean}}
- Mean of lat_plus_long for test data : {{test_lat_plus_long_mean}}
</b></font>

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [10]:
model_1_features_train = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']]
model_2_features_train = model_1_features_train.join(train_data['bed_bath_rooms']) 
model_3_features_train = model_2_features_train.join([train_data['bedrooms_squared'], train_data['log_sqft_living'],
                                         train_data['lat_plus_long']])

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [11]:
# Creating 3 Linear Regression Models
model_1 = LinearRegression().fit(model_1_features_train, train_data['price'])
model_2 = LinearRegression().fit(model_2_features_train, train_data['price'])
model_3 = LinearRegression().fit(model_3_features_train, train_data['price'])

In [12]:
# Printing the coefficients and intercepts of the 3 Linear Regression Models
print('Model_1 Summary')
print('Coefficients : ', model_1.coef_)
print('Intercept : ', model_1.intercept_)

print('\nModel_2 Summary')
print('Coefficients : ', model_2.coef_)
print('Intercept : ', model_2.intercept_)

print('\nModel_3 Summary')
print('Coefficients : ', model_3.coef_)
print('Intercept : ', model_3.intercept_)

# The code below is used to answer the following quiz questions

model1_bathroom_coef_sign = 'positive (+)' if model_1.coef_[2] > 0 else 'negative (-)'
model2_bathroom_coef_sign = 'positive (+)' if model_2.coef_[2] > 0 else 'negative (-)'

Model_1 Summary
Coefficients :  [ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05]
Intercept :  -69075726.79256986

Model_2 Summary
Coefficients :  [ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04]
Intercept :  -66867968.871078916

Model_3 Summary
Coefficients :  [ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
 -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -1.29366480e+06
  1.27334900e+05]
Intercept :  -62036084.986098416


<font color = 'steelblue'><b> Quiz 1: What is the sign (positive or negative) for the coefficient/weight for `bathrooms` in model 1?**

<font color = 'mediumvioletred'><b> Answer 1 : The sign of the coefficient for bathrooms in model 1 is {{model1_bathroom_coef_sign}} </b></font>

<br/>

<font color = 'mediumvioletred'><b> Quiz 2: What is the sign (positive or negative) for the coefficient/weight for `bathrooms` in model 2?**

<font color = 'steelblue'><b> Answer 2 : The sign of the coefficient for bathrooms in model 2 is {{model2_bathroom_coef_sign}} </b></font>

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [13]:
RSS_model_1_train_data = get_residual_sum_of_squares(model_1, model_1_features_train, train_data['price'])
RSS_model_2_train_data = get_residual_sum_of_squares(model_2, model_2_features_train, train_data['price'])
RSS_model_3_train_data = get_residual_sum_of_squares(model_3, model_3_features_train, train_data['price'])

print('RSS for model 1 on train data : ' , RSS_model_1_train_data)
print('RSS for model 2 on train data : ' , RSS_model_2_train_data)
print('RSS for model 3 on train data : ' , RSS_model_3_train_data)

RSS_list = [RSS_model_1_train_data, RSS_model_2_train_data,RSS_model_3_train_data]
model_names = ['Model 1', 'Model 2', 'Model 3']
rss_dict = dict(zip(model_names, RSS_list))

# The code below is used to answer the following quiz questions

name_lowest_RSS = min(rss_dict, key=rss_dict.get)

print('\n' + str(name_lowest_RSS) + ' has the lowest RSS on the TRAINING DATA')

RSS for model 1 on train data :  967879963049546.4
RSS for model 2 on train data :  958419635074068.2
RSS for model 3 on train data :  903436455050477.2

Model 3 has the lowest RSS on the TRAINING DATA


<font color = 'steelblue'><b> Quiz Question : Which model (1, 2 or 3) has lowest RSS on TRAINING Data? Is this what you expected? </b></font>

<font color = 'mediumvioletred'><b>Answer : {{name_lowest_RSS}} has the lowest RSS on TRAINING DATA </b></font>

Now compute the RSS on on TEST data for each of the three models.

In [14]:
# Create the features for TEST DATA
model_1_features_test = test_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']]
model_2_features_test = model_1_features_test.join(test_data['bed_bath_rooms']) 
model_3_features_test = model_2_features_test.join([test_data['bedrooms_squared'], test_data['log_sqft_living'],
                                         test_data['lat_plus_long']])

# Calculate the RSS for the models on the TEST DATA
RSS_model_1_test_data = get_residual_sum_of_squares(model_1, model_1_features_test, test_data['price'])
RSS_model_2_test_data = get_residual_sum_of_squares(model_2, model_2_features_test, test_data['price'])
RSS_model_3_test_data = get_residual_sum_of_squares(model_3, model_3_features_test, test_data['price'])

print('RSS for model 1 on test data : ' , RSS_model_1_test_data)
print('RSS for model 2 on test data : ' , RSS_model_2_test_data)
print('RSS for model 3 on test data : ' , RSS_model_3_test_data)

# The code below is used to answer the following quiz question

RSS_list = [RSS_model_1_test_data, RSS_model_2_test_data,RSS_model_3_test_data]
model_names = ['Model 1', 'Model 2', 'Model 3']
rss_dict = dict(zip(model_names, RSS_list))

name_lowest_RSS = min(rss_dict, key=rss_dict.get)

print('\n' + str(name_lowest_RSS) + ' has the lowest RSS on the TESTING Data')

RSS for model 1 on test data :  225500469795490.2
RSS for model 2 on test data :  223377462976466.7
RSS for model 3 on test data :  259236319207175.75

Model 2 has the lowest RSS on the TESTING Data


<font color = 'steelblue'><b> Quiz : Which model (1, 2 or 3) has lowest RSS on TESTING Data? Is this what you expected? Think about the features that were added to each model from the previous </b></font>

<font color = 'mediumvioletred'><b>Answer : {{name_lowest_RSS}} has the lowest RSS on TRAINING DATA </b></font>